In [1]:
import pandas as pd

In [2]:
poke = pd.read_csv('pokedex.csv', delimiter=';')
poke.head(1)

,POKEID,NAME,TYPE,TOTAL,HP,ATTACK,DEFENSE,SP_ATK,SP_DEF,SPEED,ICONS
0,1,Bulbasaur,"['Grass', 'Poison']",318,45,49,49,65,65,45,['https://img.pokemondb.net/sprites/scarlet-vi...


In [3]:
poke['TYPE'] = poke['TYPE'].apply(lambda x: x.strip("[]").replace("'", "") if pd.notna(x) else None)
poke[['TYPE', 'TYPE_2']] = poke['TYPE'].str.split(',', expand=True)
poke.insert(3, 'TYPE_2', poke.pop('TYPE_2'))
poke['TYPE_2'] = poke['TYPE_2'].fillna(poke['TYPE'])
poke.drop(columns=['ICONS'], inplace=True)

In [11]:
def get_weaknesses(row):
    weaknesses = []  

    # Define the weakness based on Pokemon types
    weakness_dict = {
        'Normal': ['Fighting'],
        'Fire': ['Water', 'Rock', 'Ground'],
        'Water': ['Electric', 'Grass'],
        'Electric': ['Ground'],
        'Grass': ['Fire', 'Ice', 'Poison', 'Flying', 'Bug'],
        'Ice': ['Fire', 'Fighting', 'Rock', 'Steel'],
        'Fighting': ['Flying', 'Psychic', 'Fairy'],
        'Poison': ['Ground', 'Psychic'],
        'Ground': ['Water', 'Ice', 'Grass'],
        'Flying': ['Electric', 'Ice', 'Rock'],
        'Psychic': ['Bug', 'Ghost', 'Dark'],
        'Bug': ['Fire', 'Flying', 'Rock'],
        'Rock': ['Water', 'Grass', 'Fighting', 'Ground', 'Steel'],
        'Ghost': ['Ghost', 'Dark'],
        'Dragon': ['Ice', 'Dragon', 'Fairy'],
        'Dark': ['Fighting', 'Bug', 'Fairy'],
        'Steel': ['Fire', 'Fighting', 'Ground'],
        'Fairy': ['Poison', 'Steel'],
    }

    # Extract types from the row
    type_1 = row['TYPE']
    type_2 = row['TYPE_2']

    if type_1 in weakness_dict:
        weaknesses.extend(weakness_dict[type_1])
    if type_2 in weakness_dict:
        weaknesses.extend(weakness_dict[type_2])

    return ','.join(weaknesses)

poke['Weaknesses'] = poke.apply(get_weaknesses, axis=1)

max_weaknesses = poke['Weaknesses'].str.count(',').max() + 1
poke[['Weakness_' + str(i+1) for i in range(max_weaknesses)]] = poke['Weaknesses'].str.split(',', expand=True)

In [13]:
poke

,POKEID,NAME,TYPE,TYPE_2,TOTAL,HP,ATTACK,DEFENSE,SP_ATK,SP_DEF,...,Weakness_1,Weakness_2,Weakness_3,Weakness_4,Weakness_5,Weakness_6,Weakness_7,Weakness_8,Weakness_9,Weakness_10
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,Fire,Ice,Poison,Flying,Bug,None,None,None,None,None
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,Fire,Ice,Poison,Flying,Bug,None,None,None,None,None
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,Fire,Ice,Poison,Flying,Bug,None,None,None,None,None
3,3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,...,Fire,Ice,Poison,Flying,Bug,None,None,None,None,None
4,4,Charmander,Fire,Fire,309,39,52,43,60,50,...,Water,Rock,Ground,Water,Rock,Ground,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,1023,Iron Crown,Steel,Psychic,590,90,72,100,122,108,...,Fire,Fighting,Ground,None,None,None,None,None,None,None
1211,1024,Terapagos Normal Form,Normal,Normal,450,90,65,85,65,85,...,Fighting,Fighting,None,None,None,None,None,None,None,None
1212,1024,Terapagos Terastal Form,Normal,Normal,600,95,95,110,105,110,...,Fighting,Fighting,None,None,None,None,None,None,None,None
1213,1024,Terapagos Stellar Form,Normal,Normal,700,160,105,110,130,110,...,Fighting,Fighting,None,None,None,None,None,None,None,None


Need to adjust logic to take into account weaknesses and not offer a counter team but your team. Good starting point... 

In [20]:
def pick_favorite_pokemon():
    print("Choose up to 4 of your favorite Pokémon:")
    favorites = []

    for i in range(4):
        pokemon = input(f"Enter the name of your favorite Pokémon {i + 1} (or type 'done' to finish): ").capitalize()
        if pokemon.lower() == 'done':
            break
        favorites.append(pokemon)

    return favorites

def suggest_counter_team(favorite_pokemon, dataframe):
    counter_team = set()

    for favorite in favorite_pokemon:
        name_column = dataframe.columns[dataframe.columns.str.lower().str.contains('name')].tolist()
        type_column = dataframe.columns[dataframe.columns.str.lower().str.contains('type')].tolist()
        attack_column = dataframe.columns[dataframe.columns.str.lower().str.contains('attack')].tolist()
        speed_column = dataframe.columns[dataframe.columns.str.lower().str.contains('speed')].tolist()

        selected_data = dataframe[dataframe[name_column[0]].str.capitalize() == favorite]

        if not selected_data.empty:
            types = selected_data[type_column[0]].tolist()
            attack = selected_data[attack_column[0]].values[0]
            speed = selected_data[speed_column[0]].values[0]

            strong_against = dataframe[
                (dataframe[type_column[1]].isin(types) | dataframe[type_column[0]].isin(types)) &
                (dataframe[name_column[0]] != favorite)  
            ]

            strong_against = strong_against.sort_values(by=[attack_column[0], speed_column[0]], ascending=False).head(8)

            counter_team.update(strong_against[name_column[0]].tolist())

    return list(counter_team)[:8]

if __name__ == "__main__":
    user_favorites = pick_favorite_pokemon()

    if not user_favorites:
        print("No favorite Pokémon selected. Exiting.")
    else:
        recommended_team = suggest_counter_team(user_favorites, poke)
        print("\nRecommended Counter Team:")
        for pokemon in recommended_team:
            print(f"- {pokemon}")

Choose up to 4 of your favorite Pokémon:

Recommended Counter Team:
- Greninja Ash-Greninja
- Charizard Mega Charizard X
- Darmanitan Standard Mode
- Gyarados Mega Gyarados
- Sharpedo Mega Sharpedo
- Kyogre Primal Kyogre
- Flareon
- Blaziken Mega Blaziken


In [21]:
poke.head().to_csv('exp.csv')

In [22]:
def suggest_pokemon_team(dataframe):
    # Sort Pokémon by total stats in descending order
    sorted_pokemon = dataframe.sort_values(by='TOTAL', ascending=False)

    # Initialize an empty team
    pokemon_team = []

    # Keep track of types already selected
    selected_types = set()

    # Iterate through sorted Pokémon to build a balanced team
    for _, pokemon in sorted_pokemon.iterrows():
        # Check if the type is not already selected and add it to the team
        if pokemon['TYPE'] not in selected_types:
            pokemon_team.append(pokemon['NAME'])
            selected_types.add(pokemon['TYPE'])

        # Check if the team size is 6
        if len(pokemon_team) == 6:
            break

    return pokemon_team

if __name__ == "__main__":
    recommended_team = suggest_pokemon_team(poke)
    print("\nRecommended Pokémon Team:")
    for pokemon in recommended_team:
        print(f"- {pokemon}")



Recommended Pokémon Team:
- Eternatus Eternamax
- Rayquaza Mega Rayquaza
- Mewtwo Mega Mewtwo X
- Groudon Primal Groudon
- Kyogre Primal Kyogre
- Arceus
